In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle as pkl
import time
import random
import re
import json
import geopandas as gpd
from SPARQLWrapper import SPARQLWrapper, JSON
from shapely.geometry import Point
import math

In [2]:
# Define the URL of the webpage with the information
# url = "https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/ajax/yw1"
url = "https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1"

In [3]:
# Send a GET request and parse the HTML content
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
# }
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "de,en-US;q=0.7,en;q=0.3",  
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "cross-site", 
    "Sec-Fetch-User": "?1", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0"
}

In [151]:
# List for row collection
player_data = []
soup_old = None
# Extract the player information from the table
for p in range(60): #there are about 50 pages for now but the exact number will change, so try to load until fail
    # print(p, 'waiting')
    time.sleep(random.uniform(1, 2))
    if p==0: page = ""
    else: page = f"/page/{p+1}"
    start_time = time.time()
    try:
        response = requests.get(url+page, headers=headers)
        # process the response here
    except requests.exceptions.RequestException:
        print(f"Error at {url+page}")
        break
    elapsed_time = time.time() - start_time
    print(f"Request of {url+page} took {elapsed_time} seconds")

    soup = BeautifulSoup(response.content, "html.parser")
    if soup_old == soup:
        print("End of pages reached") #if the same page is loaded twice, we reached the end
        break
    soup_old = soup
    

    # Find the table with the player data
    player_table = soup.find("table", class_="items")
    
    # for each row get the relevant data. Only every 4. row holds the full player data
    for r,row in enumerate(player_table.find_all("tr")[1::7]):
        player_info = {}
        columns = row.find_all("td")

        player_info["player_image_link"] = columns[1].find("img")["data-src"]
        player_info["player_name"] = columns[2].find("a")["title"]
        player_info["player_profile_link"] = "https://www.transfermarkt.ch"+columns[2].find("a")["href"]
        player_info["player_position"] = columns[3].text.strip()
        player_info["player_birthday"] = columns[8].text.strip()
        player_info["player_nationality"] = columns[10].find("img")["alt"]
        player_info["player_left"] = columns[12].text.strip()
        player_info["player_team"] = columns[14].find("a").text.strip()
        
        player_data.append(player_info)

player_data = pd.DataFrame(player_data)
player_data = player_data.drop_duplicates().reset_index(drop=True)


Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1 took 13.035407066345215 seconds
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1/page/2 took 4.206685543060303 seconds
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1/page/3 took 3.864579200744629 seconds
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1/page/4 took 4.504663705825806 seconds
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1/page/5 took 4.485477685928345 seconds
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1/page/6

In [152]:
# Save the data as a pickle file
with open("../Data/player_data.pkl", "wb") as f:
    pkl.dump(player_data, f)

In [25]:
# Load the data from the pickle file
with open("../Data/player_data.pkl", "rb") as f:
    player_data = pkl.load(f)

In [153]:
player_data

,player_image_link,player_name,player_profile_link,player_position,player_birthday,player_nationality,player_left,player_team
0,https://img.a.transfermarkt.technology/portrai...,Denis Zakaria,https://www.transfermarkt.ch/denis-zakaria/pro...,Defensives Mittelfeld,20.11.1996,Schweiz,01.07.2017,1.
1,https://img.a.transfermarkt.technology/portrai...,Djibril Sow,https://www.transfermarkt.ch/djibril-sow/profi...,Zentrales Mittelfeld,06.02.1997,Schweiz,01.07.2019,1.
2,https://img.a.transfermarkt.technology/portrai...,Seydou Doumbia,https://www.transfermarkt.ch/seydou-doumbia/pr...,Mittelstürmer,31.12.1987,Elfenbeinküste,01.07.2010,1.
3,https://img.a.transfermarkt.technology/portrai...,Josef Martínez,https://www.transfermarkt.ch/josef-martinez/pr...,Mittelstürmer,19.05.1993,Venezuela,01.07.2014,1.
4,https://img.a.transfermarkt.technology/portrai...,Miralem Sulejmani,https://www.transfermarkt.ch/miralem-sulejmani...,Linkes Mittelfeld,05.12.1988,Serbien,01.07.2022,1.
...,...,...,...,...,...,...,...,...
1230,https://img.a.transfermarkt.technology/portrai...,? Scheidegger,https://www.transfermarkt.ch/-scheidegger/prof...,Mittelfeld,-,Schweiz,-,1.
1231,https://img.a.transfermarkt.technology/portrai...,Martin Bircher,https://www.transfermarkt.ch/martin-bircher/pr...,Mittelfeld,-,Schweiz,-,1.
1232,https://img.a.transfermarkt.technology/portrai...,Etienne Moser,https://www.transfermarkt.ch/etienne-moser/pro...,Torwart,02.07.1995,Schweiz,-,Jugend
1233,https://img.a.transfermarkt.technology/portrai...,Matteo Blumenthal,https://www.transfermarkt.ch/matteo-blumenthal...,Abwehr,13.03.1996,Schweiz,-,Jugend


#### Reduce Players

In [27]:
# select only players that played in the first team
selection = player_data[player_data["player_team"]=="1."]
# extract the date from player_left
# first sort out entries with "-"
selection = selection[selection["player_left"]>"-"]
# then replace all entries that contain vsl. bis with None (not left yet)
selection["player_left"] = selection["player_left"].apply(lambda x: None if "vsl. bis" in x else x)
# then convert dates to datetime
selection["player_left"] = pd.to_datetime(selection["player_left"], format="%d.%m.%Y")
# then select only players that left after 2000 or are still in the team
selection = selection[(selection["player_left"]>pd.to_datetime("2000-01-01")) | (selection["player_left"].isna())]
# convert dates back to string
selection["player_left"] = selection["player_left"].apply(lambda x: x.strftime("%d.%m.%Y") if not pd.isna(x) else None)
# reset index
selection = selection.reset_index(drop=True)

In [47]:
selection.head()

,player_image_link,player_name,player_profile_link,player_position,player_birthday,player_nationality,player_left,player_team
0,https://img.a.transfermarkt.technology/portrai...,Denis Zakaria,https://www.transfermarkt.ch/denis-zakaria/pro...,Defensives Mittelfeld,20.11.1996,Schweiz,01.07.2017,1.
1,https://img.a.transfermarkt.technology/portrai...,Djibril Sow,https://www.transfermarkt.ch/djibril-sow/profi...,Zentrales Mittelfeld,06.02.1997,Schweiz,01.07.2019,1.
2,https://img.a.transfermarkt.technology/portrai...,Seydou Doumbia,https://www.transfermarkt.ch/seydou-doumbia/pr...,Mittelstürmer,31.12.1987,Elfenbeinküste,01.07.2010,1.
3,https://img.a.transfermarkt.technology/portrai...,Josef Martínez,https://www.transfermarkt.ch/josef-martinez/pr...,Mittelstürmer,19.05.1993,Venezuela,01.07.2014,1.
4,https://img.a.transfermarkt.technology/portrai...,Miralem Sulejmani,https://www.transfermarkt.ch/miralem-sulejmani...,Linkes Mittelfeld,05.12.1988,Serbien,01.07.2022,1.


### Request the transfer history of players

In [28]:
def cutClubURL(column):
    tmp = "https://www.transfermarkt.ch"+column.find("a")["href"]
    # Split the string on "/"
    split_url = tmp.split('/')
    # Get Club TM_id
    TM_id = split_url[-3]
    # Remove the last two elements
    new_url = '/'.join(split_url[:-2])
    return new_url, TM_id

# Write a function that requests the transfer history of a player based on the player profile link using the requests library
def requestTransferHistory(player_profile_link,headers):
    player_transfer = []
    start_time = time.time()
    try:
        response = requests.get(player_profile_link, headers=headers)
    except requests.exceptions.RequestException:
        print(f"Error at {player_profile_link}")
        return None
    elapsed_time = time.time() - start_time
    # print(f"Request of {player_profile_link} took {elapsed_time} seconds")  
    soup = BeautifulSoup(response.content, "html.parser")
    # Find the table with the player data
    transfer_table = soup.find_all("div", class_="tm-player-transfer-history-grid")
    # exclude rows with class="tm-player-transfer-history-grid--heading
    transfer_table = [row for row in transfer_table if "tm-player-transfer-history-grid--heading" not in row.get('class')]
    for r,row in enumerate(transfer_table):
        transfer_info = {}
        columns = row.find_all("div")
        transfer_info["season"] = columns[0].text.strip()
        transfer_info["transfer_date"] = columns[1].text.strip()
        transfer_info["team_left_name"] = columns[2].text.strip()
        temp, transfer_info["team_left_TM_id"] = cutClubURL(columns[2])
        # transfer_info["team_left_logo"] = columns[2].find("img", class_="tm-player-transfer-history-grid__club-logo")["data-srcset"]
        # transfer_info["team_left_logo"] = re.search("(?P<url>https?://[^\s]+)", transfer_info["team_left_logo"]).group("url") #extract the url from the srcset
        # try: transfer_info["team_left_flag"] = columns[2].find("img", class_="tm-player-transfer-history-grid__flag")["data-src"]
        # except: transfer_info["team_left_flag"] = None
        transfer_info["team_joined_name"] = columns[3].text.strip()
        temp, transfer_info["team_joined_TM_id"] = cutClubURL(columns[3])
        # transfer_info["team_joined_logo"] = columns[3].find("img", class_="tm-player-transfer-history-grid__club-logo")["data-srcset"]
        # transfer_info["team_joined_logo"] = re.search("(?P<url>https?://[^\s]+)", transfer_info["team_joined_logo"]).group("url") #extract the url from the srcset
        # try: transfer_info["team_joined_flag"] = columns[3].find("img", class_="tm-player-transfer-history-grid__flag")["data-src"]
        # except: transfer_info["team_joined_flag"] = None
        transfer_info["market_value"] = columns[4].text.strip()
        transfer_info["transfer_fee"] = columns[5].text.strip()
        player_transfer.append(transfer_info)
    return player_transfer

In [29]:
selection["player_transfer_history"] = None
for i,player in selection.iterrows():
    player_transfer = requestTransferHistory(player.player_profile_link,headers)
    selection["player_transfer_history"].loc[i]=player_transfer
    time.sleep(random.uniform(1, 2))
with open("../Data/transfers.pkl", "wb") as f:
    pkl.dump(selection, f)

In [7]:
# Load the data from the pickle file
with open("../Data/transfers.pkl", "rb") as f:
    selection = pkl.load(f)

In [159]:
selection.head()

,player_image_link,player_name,player_profile_link,player_position,player_birthday,player_nationality,player_left,player_team,player_transfer_history
0,https://img.a.transfermarkt.technology/portrai...,Miralem Sulejmani,https://www.transfermarkt.ch/miralem-sulejmani...,Linkes Mittelfeld,05.12.1988,Serbien,01.07.2022,1.,"[{'season': '22/23', 'transfer_date': '01.07.2..."
1,https://img.a.transfermarkt.technology/portrai...,Jordan,https://www.transfermarkt.ch/jordan/profil/spi...,Mittelstürmer,26.04.1996,Vereinigte Staaten,01.07.2022,1.,"[{'season': '22/23', 'transfer_date': '01.07.2..."
2,https://img.a.transfermarkt.technology/portrai...,Fabian Rieder,https://www.transfermarkt.ch/fabian-rieder/pro...,Offensives Mittelfeld,16.02.2002,Schweiz,None,1.,"[{'season': '20/21', 'transfer_date': '14.01.2..."
3,https://img.a.transfermarkt.technology/portrai...,Jean-Pierre Nsame,https://www.transfermarkt.ch/jean-pierre-nsame...,Mittelstürmer,01.05.1993,Kamerun,None,1.,"[{'season': '21/22', 'transfer_date': '30.06.2..."
4,https://img.a.transfermarkt.technology/portrai...,Edimilson Fernandes,https://www.transfermarkt.ch/edimilson-fernand...,Innenverteidiger,15.04.1996,Schweiz,30.06.2022,1.,"[{'season': '21/22', 'transfer_date': '30.06.2..."


In [113]:
start_year="2015"
# for now constrain to players that left after 14/15 season
# then convert dates to datetime
selection["player_left"] = pd.to_datetime(selection["player_left"], format="%d.%m.%Y")
# then select only players that left after 2000 or are still in the team
selection = selection[(selection["player_left"]>pd.to_datetime(start_year+"-07-01")) | (selection["player_left"].isna())]
# convert dates back to string
selection["player_left"] = selection["player_left"].apply(lambda x: x.strftime("%d.%m.%Y") if not pd.isna(x) else None)

In [114]:
out = selection.to_dict(orient="records")
with open(f"../Data/transfers.json", "w") as file:
    json.dump(out, file)

### List of Seasons

In [ ]:
seasons_dict = {}

active = False
for p, player in selection.iterrows():
    for transfer in player.player_transfer_history:
        season = transfer["season"]

        if transfer["team_left_TM_id"] == "452":
            # Ensure the season exists in our dictionary
            if season not in seasons_dict:
                seasons_dict[season] = {'players_left': [], 'players_joined': [], 'players_const': []}         
            seasons_dict[season]['players_left'].append(player.to_dict())
            active = True
        elif active:
            # Ensure the season exists in our dictionary
            if season not in seasons_dict:
                seasons_dict[season] = {'players_left': [], 'players_joined': [], 'players_const': []}
            seasons_dict[season]['players_const'].append(player.to_dict())
        if transfer["team_joined_TM_id"] == "452":
            # Ensure the season exists in our dictionary
            if season not in seasons_dict:
                seasons_dict[season] = {'players_left': [], 'players_joined': [], 'players_const': []}       
            seasons_dict[season]['players_joined'].append(player.to_dict())
            active = False

# Convert the dictionary to a DataFrame
seasons = pd.DataFrame.from_dict(seasons_dict, orient='index').reset_index()
seasons = seasons.rename(columns={"index": "season"})
seasons = seasons.sort_values(by=['season']).reset_index(drop=True)
# filter seasons with XX/YY format where XX >= 15
seasons = seasons[seasons['season'].str[:2].astype(int) >= int(start_year[-2:])]


In [153]:
player.to_dict()

{'player_image_link': 'https://img.a.transfermarkt.technology/portrait/medium/default.jpg?lm=1',
 'player_name': 'Malik Deme',
 'player_profile_link': 'https://www.transfermarkt.ch/malik-deme/profil/spieler/805772',
 'player_position': 'Rechtsaußen',
 'player_birthday': '16.03.2005',
 'player_nationality': 'Schweiz',
 'player_left': None,
 'player_team': '1.',
 'player_transfer_history': [{'season': '23/24',
   'transfer_date': '01.07.2023',
   'team_left_name': 'GC Zürich U21',
   'team_left_TM_id': '5502',
   'team_joined_name': 'BSC Young Boys',
   'team_joined_TM_id': '452',
   'market_value': '150 Tsd. €',
   'transfer_fee': 'ablösefrei'},
  {'season': '22/23',
   'transfer_date': '01.07.2022',
   'team_left_name': 'GC Zürich U18',
   'team_left_TM_id': '14543',
   'team_joined_name': 'GC Zürich U21',
   'team_joined_TM_id': '5502',
   'market_value': '-',
   'transfer_fee': '-'},
  {'season': '20/21',
   'transfer_date': '01.07.2020',
   'team_left_name': 'GC Jugend',
   'team_le

In [ ]:
out = seasons.to_dict(orient="records")
with open(f"../Data/seasons.json", "w") as file:
    json.dump(out, file)

### List of clubs

In [143]:
# This code iterates through every player in the selection dataframe, and then iterates through each transfer in the player's transfer history. It then checks if the name of the team that the player joined/left is in the clubs dataframe. If not, it adds the team to the clubs dataframe.

clubs=pd.DataFrame({'name': [], 'id': [], 'players': []})
for p,player in selection.iterrows():
    for transfer in player.player_transfer_history:
        if transfer["team_joined_TM_id"] not in clubs.id.values:
            clubs=clubs.append({'name': transfer["team_joined_name"],
                                'id': transfer["team_joined_TM_id"],
                                'players': [player.player_name]}, ignore_index=True)
        elif player.player_name not in clubs[clubs.id==transfer["team_joined_TM_id"]].players.values[0]:
                clubs.loc[clubs.id==transfer["team_joined_TM_id"], "players"].values[0].append(player.player_name)
        if transfer["team_left_TM_id"] not in clubs.id.values:
            clubs=clubs.append({'name': transfer["team_left_name"],
                                'id': transfer["team_left_TM_id"],
                                'players': [player.player_name]}, ignore_index=True)
        elif player.player_name not in clubs[clubs.id==transfer["team_left_TM_id"]].players.values[0]:
                clubs.loc[clubs.id==transfer["team_left_TM_id"], "players"].values[0].append(player.player_name)
# remove clubs juniour clubs
clubs=clubs[~clubs.name.str.contains("U\d\d|Jgd.|B$|II$|Jugend|Jg$|Juniors$")]
# Handeling renaming of clubs that lead to new TM_id
clubs.loc[clubs.id=="5967","id"] = "31404" #Visby IF Gute -> FC Gute


# sum up the number of players per club
clubs.loc[:,"players"] = clubs.loc[:,"players"].apply(lambda x: len(x))

/tmp/ipykernel_586/3390669855.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if transfer["team_joined_TM_id"] not in clubs.id.values:


In [144]:
# clubs that have the same id
clubs[clubs.duplicated(subset=["id"], keep=False)]

,name,id,players


### Get club coordinates

In [145]:
# Create SPARQL client
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
data = []

i=0
while i < len(clubs):
  # SPARQL Query
  query = """
  SELECT ?ID ?clubLabel (SAMPLE(?coord) as ?coordinates) WHERE {
  
    ?club wdt:P7223 ?ID .
    FILTER(?ID in """ + str(tuple(clubs.iloc[i:i+30]["id"])) + """)
    # Try to get location of stadium
    OPTIONAL {
      ?club wdt:P115 ?home1 .
      ?home1 wdt:P625 ?coord1 .
    }
    # Else get it from the city
    OPTIONAL {
      ?club wdt:P159 ?home2 .
      ?home2 wdt:P625 ?coord2 .
    }
    #If first one is not bound take second one
    BIND(COALESCE(?coord1, ?coord2) AS ?coord)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    
  } GROUP BY ?ID ?clubLabel #some stadiums have multiple locations -> choose only one (at random)
  """
  # Query for capitals and their coordinates
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)

  # Send request
  results = sparql.query().convert()

  # Extract results
  for result in results["results"]["bindings"]:
      # Parse point string
      point_str = result['coordinates']['value'].replace('Point(', '').replace(')', '')
      lon, lat = map(float, point_str.split())
      point = Point(lon, lat)
      
      data.append({
          'id': result['ID']['value'],
          'coordinates': point,
      })
  i+=30

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry='coordinates')

**"Unbekannt", "Vereinslos", und "Karriereende" need to be assigned coordinates manually**

In [146]:
clubs = clubs.merge(gdf, left_on="id", right_on="id", how="left")

### Add Coordinates to Unbekannt, Karriereende, Vereinslos
clubs.loc[clubs.name=="Unbekannt", "coordinates"] = Point(-8.4, 46)
clubs.loc[clubs.name=="Karriereende", "coordinates"] = Point(-8.4, 48,)
clubs.loc[clubs.name=="Vereinslos", "coordinates"] = Point(-8.4, 47)

In [147]:
# show all clubs without coordinates
clubs[clubs.coordinates.isna()]

,name,id,players,coordinates


In [148]:
# clubs that have the same id
clubs[clubs.duplicated(subset=["id"], keep=False)]

,name,id,players,coordinates


Es gibt:
* Vereine auf Wikidata aber ohne TM ID (+ manchmal ohne coordinaten/Ortsnamen)
* Vereinsnamen ohne Ort (zumindest gekürzter name)
* Falsche Vereinsnamen auf TM richtiger auf wikidata: z.b. Balmazújváros(i)
* Vereine ohne alles (Location nur mit manueller recherche auffindbar)


**I chose to add missing data to wikidata by hand, because this way I am not only helping myself but also giving something back to the community**

**Now I need to shift coordinates slightly wehre clubs play at the same location**

In [149]:
RADIUS_LAT_LNG_DEGREES = 0.00002

def separate_clustered_geolocations(clubs):
    out_clubs = clubs.copy(deep=True)
    for i, group in out_clubs[out_clubs.duplicated(subset="coordinates", keep=False)].groupby("coordinates", sort=False):
        group_size = len(group)

        # itereate over each club in the group
        count = 0
        for index, row in group.iterrows():
            angle = count / float(group_size) * 2.0 * math.pi
            # assign shifted location to the club
            out_clubs.loc[index, "coordinates"] = shift_location(row['coordinates'], angle)
            count += 1
    return out_clubs


def shift_location(point, angle):
    # Calculate radius for lon lat degrees
    #RADIUS_LAT_LNG_DEGREES = 0.00002
    radiusKm = 0.5
    radiusLon = 1 / (111.319 * math.cos(point.y * (math.pi / 180))) * radiusKm
    radiusLat = 1 / 110.574 * radiusKm   
    # shift the location by a small amount (around a circle) to avoid overlapping points
    return Point(point.x + radiusLon * math.cos(angle), point.y + radiusLat * math.sin(angle))

In [150]:
clubs = separate_clustered_geolocations(clubs)

In [152]:
# make geodataframe from clubs
geo_clubs = gpd.GeoDataFrame(clubs, geometry='coordinates')
geo_clubs.to_file("../Data/clubs.geojson", driver="GeoJSON")

Link to club image: https://tmssl.akamaized.net/images/wappen/head/{teamID}.png

Next steps:
* Bar with seasons (need to think about how (to what season) to count arrivals and leaves)
* Vizualize leaves (first for one season, then for all) Thiker if more than one?; Make sure that youth clubs are handled
* Switch for Ab/Zugänge * viz zugänge

* Handle Clubs with no marker
* Only display clubs relevant for view